# Importing Required Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install contractions
# apt install python3-contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 14.9 MB/s eta 0:00:00


In [3]:
!pip install scikit-learn

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.layers import Dense
import json
import re
import csv
import string
import json

from sklearn.feature_extraction.text import TfidfVectorizer
import unicodedata
from sklearn.model_selection import train_test_split


import contractions
#

# Read a dialog file and create a DataFrame of questions and answers.

In [5]:

question  =[]
answer = []
# Opening JSON file
f = open('/content/drive/MyDrive/data-5.json')

# returns JSON object as
# a dictionary
data = json.load(f)

# Iterating through the json
# list
i=0
for i in range(len(data)):
    # i+=1
    # if i==6:
    #         break
    for j in data[i]["Q&As"]:
#         print(type(list(j.values())))
        question.append(list(j.values())[0])
        answer.append(list(j.values())[1])
data = pd.DataFrame({"question" : question ,"answer":answer})
data.head()
# Closing file
# f.close()

,question,answer
0,Why is it important to support BIPOC owned fas...,Supporting BIPOC owned fashion brands is impor...
1,Why is it challenging to find BIPOC owned fash...,It is challenging to find BIPOC owned fashion ...
2,What is the significance of featuring BIPOC ow...,Featuring BIPOC owned brands in content is sig...
3,How can supporting BIPOC owned brands contribu...,Supporting BIPOC owned brands is a tangible wa...
4,What is the message conveyed by the unique col...,The unique collections of BIPOC owned brands c...


# Preprocessing text data using unicode normalization, regular expressions, and string manipulation

unicode_to_ascii: This function converts Unicode strings to
ASCII, removing diacritics (accents) from characters. This simplification can help reduce the complexity of the dataset for natural language processing.

clean_text: This function performs several preprocessing steps on the text: converting to lowercase, removing numbers, expanding contractions, cleaning up various English expressions and removing punctuation. It also wraps sentences with <sos> (start of sentence) and <eos> (end of sentence) tokens, which are common practices in sequence modeling to indicate the beginning and end of sentences.




In [6]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')

def clean_text(text):
    text = unicode_to_ascii(text.lower().strip())
    text = " ".join(text.split())
    text = ''.join([i for i in text if not i.isdigit()])
    contractions.fix(text)
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"hi", "hey", text)
    text = re.sub(r"\r", "", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub("(\\W)"," ",text)
    text = re.sub('\S*\d\S*\s*','', text)
    text =  "<sos> " +  text + " <eos>"
    return text

#

# Clean, Convert, & Tokenize the data

This cell applies the clean_text function to the questions and answers to prepare them for the model. It then tokenizes the cleaned text, converting each word to a numerical index. Tokenization is crucial for neural networks since they require numerical input. The data is also padded to ensure that all sequences are the same length, which is necessary for batch processing.

In [7]:
data["question"] = data.question.apply(clean_text)
data["answer"] = data.answer.apply(clean_text)
question  = data.question.values.tolist()
answer =  data.answer.values.tolist()

def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)
    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    return tensor, lang_tokenizer
print(len(data["question"]))

3945


# Setting up data for sequence-to-sequence model training!

Here, we perform various data preprocessing steps like tokenization, removal of start and end tags, and creation of training and validation datasets using an 80-20 split. The tokenization process involves creating word-to-index and index-to-word dictionaries for both input and target languages. We can also set various hyperparameters for the model like the embedding dimension, number of units, vocabulary size, batch size, and buffer size.`

In [8]:
input_tensor , inp_lang  =  tokenize(question)
target_tensor , targ_lang  =  tokenize(answer)

def remove_tags(sentence):
    return sentence.split("<start>")[-1].split("<end>")[0]

max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 1
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim =256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

print((input_tensor[0][3]) , inp_lang ,target_tensor , targ_lang)

28 <keras.src.preprocessing.text.Tokenizer object at 0x7be0c0c23580> [[   3  116 1702 ...    0    0    0]
 [   3   20    9 ...    0    0    0]
 [   3 1248 1702 ...    0    0    0]
 ...
 [   3  750  746 ...    0    0    0]
 [   3  750  746 ...    0    0    0]
 [   3  750  746 ...    0    0    0]] <keras.src.preprocessing.text.Tokenizer object at 0x7be0a6532320>


# Encoder class for Sequence-to-Sequence model

Defines the Encoder class as part of the Seq2Seq model. It uses an Embedding layer to convert token indices into dense vectors of fixed size and a GRU layer to process the sequence input. The GRU (Gated Recurrent Unit) is a type of RNN (Recurrent Neural Network) architecture.

In [9]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x,hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

# Encoder Model Initialization and Output Shape

This cell instantiates the Encoder with the specified vocabulary size, embedding dimension, units, and batch size. It initializes the hidden state and then passes a sample input batch through the encoder to test it. The shapes of the output and hidden state are printed to verify the dimensions, which is essential for debugging and ensuring the model components interact as expected.

In [10]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (1, 30, 1024)
Encoder Hidden state shape: (batch size, units) (1, 1024)


# Bahdanau Attention Layer Implementation for Neural Machine Translation

The code defines a custom Keras layer that implements Bahdanau attention mechanism for Neural Machine Translation. Bahdanau attention is a type of attention mechanism that computes attention weights over the input sequence at every time step, and generates a context vector that is a weighted sum of the input sequence. This context vector is then used by the decoder to generate the output sequence. The implementation uses two dense layers, W1 and W2, to map the input and query to a common hidden space, and a dense layer V to generate the attention weights. The layer takes as input the query (decoder hidden state) and the values (encoder output sequence), and outputs the context vector and attention weights.`

In [11]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        # query hidden state shape == (batch_size, hidden size)
        # query_with_time_axis shape == (batch_size, 1, hidden size)
        # values shape == (batch_size, max_len, hidden size)
        # we are doing this to broadcast addition along the time axis to calculate the score
        query_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(
            self.W1(query_with_time_axis) + self.W2(values)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

#         print("attention_weights : ",attention_weights)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

# Applying Bahdanau Attention to Encoder Outputs

Here, an instance of the BahdanauAttention class is created and tested with sample data. This piece of code demonstrates how the attention mechanism can be applied to the hidden state and output of the encoder, showing the dimensions of the resulting context vector and attention weights.

In [12]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (1, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (1, 30, 1)


#A decoder class that takes the output from the encoder and generates the target sequence word by word!

The Decoder class is a sub-class of the tf.keras.Model class, which defines the decoding part of the Sequence-to-Sequence model used for the chatbot. The decoder is responsible for generating the output sequence given the encoded input sequence and the previous state.

In [13]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

        # used for attention
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        context_vector, attention_weights = self.attention(hidden, enc_output)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state, attention_weights

# Decoder model initialization and sample output

This cell initializes the decoder with the specified parameters and tests it using a sample input, a hidden state, and encoder output. The shape of the decoder output is printed to ensure it's correct; this output represents the probability distribution over the target vocabulary for each batch item.

In [14]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (1, 8650)


# Loss function using Adam optimizer

Sets up the optimizer and loss function for training the Seq2Seq model. The Adam optimizer is used for its efficiency with large datasets and parameter spaces. The loss function is defined to handle sequence prediction, ignoring padding tokens in the loss calculation to prevent them from affecting the model's learning.

In [15]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
#     print("real : ",real,"\npredicted : ",pred)
#     print ("loss_ : ",loss_)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

#

# Training Step Function using Teacher Forcing

In [16]:
@tf.function
def train_step(inp, targ, enc_hidden,variables=np.array([]),gradients=np.array([])):#,variables=np.array([])
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)

        dec_hidden = enc_hidden

        dec_input = tf.expand_dims([targ_lang.word_index['<sos>']] * BATCH_SIZE, 1)

        # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]):
            # passing enc_output to the decoder
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

            loss += loss_function(targ[:, t], predictions)
#             print("loss : ",loss)
            # using teacher forcing
            dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))

#     if len(variables)<=0:
    variables = encoder.trainable_variables + decoder.trainable_variables
#     print("variables : ",variables)
#     print("vrb : ",vrb)
    gradients = tape.gradient(loss, variables)

#     print("gradients : ",gradients)

    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss, variables,gradients

#Training loop for the Chatbot Model


The training loop iterates over the dataset for a number of epochs, calling the training step function for each batch. It often tracks the loss and sometimes other metrics, printing them out or logging them for monitoring. This loop is where the actual learning occurs, as the model's weights are adjusted to minimize the loss on the training data.
It appears that the loss decreases with each epoch, which suggests that the model is improving and learning to generate more accurate translations. The decreasing loss values are also a good sign that the optimizer and loss function are appropriate for the task at hand.`

In [17]:
# Run it first time
vrb=grd=[]

In [29]:
# run it other times
vrb=pd.read_pickle("/content/drive/MyDrive/vrb.pkl")
grd=pd.read_pickle("/content/drive/MyDrive/grd.pkl")
# pd.read_pickle("grd.npy")

In [18]:
EPOCHS = 50

for epoch in range(1, EPOCHS + 1):
    print(epoch)
    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss,vrb,grd = train_step(inp, targ, enc_hidden,vrb,grd)#,vrb,grd
        total_loss += batch_loss
    df1=pd.Series(vrb)
    df1.to_pickle("vrb.pkl")

    df2=pd.DataFrame(grd)
    df2.to_pickle("grd.pkl")
    print('Epoch:{:3d} Loss:{:.4f}'.format(epoch,
                                          total_loss / steps_per_epoch))


#         np.save(f, )

#     if(epoch % 2 == 0):



1
Epoch:  1 Loss:2.0835
2
Epoch:  2 Loss:1.9568
3
Epoch:  3 Loss:1.7613
4
Epoch:  4 Loss:1.6017
5
Epoch:  5 Loss:1.4410
6
Epoch:  6 Loss:1.2772
7
Epoch:  7 Loss:1.1160
8
Epoch:  8 Loss:0.9603
9
Epoch:  9 Loss:0.8127
10
Epoch: 10 Loss:0.6790
11
Epoch: 11 Loss:0.5611
12
Epoch: 12 Loss:0.4618
13
Epoch: 13 Loss:0.3807
14
Epoch: 14 Loss:0.3125
15
Epoch: 15 Loss:0.2607
16
Epoch: 16 Loss:0.2204
17
Epoch: 17 Loss:0.1880
18
Epoch: 18 Loss:0.1647
19
Epoch: 19 Loss:0.1453
20
Epoch: 20 Loss:0.1312
21
Epoch: 21 Loss:0.1212
22
Epoch: 22 Loss:0.1116
23
Epoch: 23 Loss:0.1023
24
Epoch: 24 Loss:0.0991
25
Epoch: 25 Loss:0.0943
26
Epoch: 26 Loss:0.0894
27
Epoch: 27 Loss:0.0853
28
Epoch: 28 Loss:0.0822
29
Epoch: 29 Loss:0.0798
30
Epoch: 30 Loss:0.0775
31
Epoch: 31 Loss:0.0740
32
Epoch: 32 Loss:0.0715
33
Epoch: 33 Loss:0.0704
34
Epoch: 34 Loss:0.0690
35
Epoch: 35 Loss:0.0660
36
Epoch: 36 Loss:0.0651
37
Epoch: 37 Loss:0.0645
38
Epoch: 38 Loss:0.0619
39
Epoch: 39 Loss:0.0594
40
Epoch: 40 Loss:0.0600
41
Epoch:

In [20]:
# With this line:
encoder.save('/content/drive/MyDrive/model_state_final', save_format='tf')

# Evaluation function for the Chatbot Model

An evaluation function is used to test the chatbot's performance on unseen data. It's similar to the training function but without the backpropagation step—no weights are updated. The function typically runs the model on a set of questions and generates responses, then compares these generated responses to the actual answers. This function helps gauge how well the model is learning and generalizing.


In [21]:
def evaluate(sentence):
    sentence = clean_text(sentence)
    try:
        inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    except:
        return "I am sorry, I dont have the appropriate answer for you at this time",sentence
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<sos>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.index_word[predicted_id] + ' '

        if targ_lang.index_word[predicted_id] == '<eos>':
            return remove_tags(result), remove_tags(sentence)

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return remove_tags(result), remove_tags(sentence)

#

# Loading and using our Chatbot model to answer questions!

In [ ]:
questions  =[]
answers = []
# with open("dialogs.txt",'r') as f :
#     for line in f :
#         line  =  line.split('\t')
#         questions.append(line[0])
#         answers.append(line[1])
# print(len(question) == len(answer))

def ask(sentence):
    result, sentence = evaluate(sentence)

    print('Question: %s' % (sentence))
    print('Predicted answer: {}'.format(result))
    return result
# ask(questions[10])
# print("Actual : ")
# print(questions[10],"\n",answers[10])

In [23]:
def ask(sentence):
    result, sentence = evaluate(sentence)
    # Remove <eos> and <sos> tags from the result
    result = result.replace('<eos>', '').replace('<sos>', '')

    print('Question: %s' % sentence)
    print('Predicted answer: {}'.format(result))
    return result

def main():
    # Path to your questions file and where you want to save the answers
    questions_file = '/content/drive/MyDrive/extracted_questions.txt'
    answers_file = '/content/drive/MyDrive/predicted_answers.txt'

    # Read all the questions from the file
    with open(questions_file, 'r') as q_file:
        questions = q_file.read().split('\n\n')  # Assuming each question is separated by a newline

    # Open the answers file for writing
    with open(answers_file, 'w') as a_file:
        # Iterate through each question and generate answers
        for question in questions:
            if question.strip():  # Check if the question is not empty
                answer = ask(question.strip())
                a_file.write(answer + '\n\n')  # Write each answer to the file, separated by newlines

if __name__ == "__main__":
    main()


Question: <sos> what is the most carbonefficient option for sheypping <eos>
Predicted answer: transporting goods via boat is the most carbonefficient option currently available a big sheyp emits about grams of carbon dioxide to transport metric ton of cargo kilometer  
Question: <sos> what are some tips for switcheyng from fast fasheyon to vintage on instagram <eos>
Predicted answer: to make it hard to break up with friends  
Question: <sos> is calling out fast fasheyon considered classist <eos>
Predicted answer: yes calling out fast fasheyon is often considered classist due to the perception that sustainable fasheyon is often considered classist due to the perception that sustainable fasheyon is often considered classist due to the perception that sustainable fasheyon is often considered classist due to the perception that sustainable fasheyon is often considered classist due to the perception that sustainable fasheyon is often considered classist due to the perception that sustainabl

In [24]:
ask('What is Louis Vuitton known for?')

Question: <sos> what is louis vuitton known for <eos>
Predicted answer: louis vuitton is known for its distinctive monogrammed accessories and clotheyng  


'louis vuitton is known for its distinctive monogrammed accessories and clotheyng  '

In [31]:
ask('What is Louis Vuitton known for?')

Question: <sos> what is louis vuitton known for <eos>
Predicted answer: louis vuitton is known for its distinctive monogrammed accessories and clotheyng  


'louis vuitton is known for its distinctive monogrammed accessories and clotheyng  '

In [30]:
ask('is fair trade is important?')

Question: <sos> is fair trade is important <eos>
Predicted answer: as it can take care bamboo rayon and resources used as one accessible and the environment by taking care after the environment and promotes sustainable option as it gives and turns them as it gives and turns them as it gives and turns them as it gives and turns them as it gives and turns them as it gives and turns them as it gives and turns them as it gives and turns them as it gives and turns them as it gives and turns them 


'as it can take care bamboo rayon and resources used as one accessible and the environment by taking care after the environment and promotes sustainable option as it gives and turns them as it gives and turns them as it gives and turns them as it gives and turns them as it gives and turns them as it gives and turns them as it gives and turns them as it gives and turns them as it gives and turns them as it gives and turns them '

In [28]:
# Define the path to your text file
file_path = '/content/drive/MyDrive/extracted_answers.txt'

# Read the file and filter out any blank lines
with open(file_path, 'r') as file:
    lines = file.readlines()
    non_blank_lines = [line for line in lines if line.strip()]

# Write the non-blank lines back to the file
with open(file_path, 'w') as file:
    file.writelines(non_blank_lines)
